In [1]:
import pandas as pd
import numpy as np
import os

-----------------------------------------
LOADING THE DOORDASH ETA DATASET
------------------------

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("dharun4772/doordash-eta-prediction")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/doordash-eta-prediction


In [3]:

files = os.listdir(path)

# 'files' contain ['historical_data.csv'] to load that csv

for file in files:
    if file.endswith(".csv"):
        dataset_path = os.path.join(path, file)
        df = pd.read_csv(dataset_path)
        print(f"Loaded data from {file}")
        break

Loaded data from historical_data.csv


----------------------------------------------
LOADED TIME SERIES DATAFRAME
-------------------------------

In [4]:
df

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0
2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,5477,NaN,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0
3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,5477,NaN,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0
4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,5477,NaN,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197423,1.0,2015-02-17 00:19:41,2015-02-17 01:24:48,2956,fast,4.0,3,1389,3,345,649,17.0,17.0,23.0,251,331.0
197424,1.0,2015-02-13 00:01:59,2015-02-13 00:58:22,2956,fast,4.0,6,3010,4,405,825,12.0,11.0,14.0,251,915.0
197425,1.0,2015-01-24 04:46:08,2015-01-24 05:36:16,2956,fast,4.0,5,1836,3,300,399,39.0,41.0,40.0,251,795.0
197426,1.0,2015-02-01 18:18:15,2015-02-01 19:23:22,3630,sandwich,1.0,1,1175,1,535,535,7.0,7.0,12.0,446,384.0


---------------------------------------------------------
Data Processing (Handling Null Values and Removing Outliers) & Feature Engineering
-----------------------------------------

----------------------------------
Checking for Null Values in Various Columns of Dataframe 'df'

In [5]:
df.isna().sum()

market_id                                         987
created_at                                          0
actual_delivery_time                                7
store_id                                            0
store_primary_category                           4760
order_protocol                                    995
total_items                                         0
subtotal                                            0
num_distinct_items                                  0
min_item_price                                      0
max_item_price                                      0
total_onshift_dashers                           16262
total_busy_dashers                              16262
total_outstanding_orders                        16262
estimated_order_place_duration                      0
estimated_store_to_consumer_driving_duration      526
dtype: int64

In [6]:
# Market_ID is checked for null values and these rows are removed

critical_columns = [
    'market_id',
    'actual_delivery_time'
]

# Removing rows with null values in the above mentioned attributes or columns
df = df.dropna(subset=critical_columns)

In [7]:
df.isna().sum()

market_id                                           0
created_at                                          0
actual_delivery_time                                0
store_id                                            0
store_primary_category                           4268
order_protocol                                    508
total_items                                         0
subtotal                                            0
num_distinct_items                                  0
min_item_price                                      0
max_item_price                                      0
total_onshift_dashers                           16194
total_busy_dashers                              16194
total_outstanding_orders                        16194
estimated_order_place_duration                      0
estimated_store_to_consumer_driving_duration      525
dtype: int64

In [8]:

def handle_null_values(df):
        
    # Store Primary Category is filled with a new category and filled as 'Uncategorized'
    df['store_primary_category'].fillna('Uncategorized', inplace=True)
    
    # Order Protocol doesn't have too many null values so I just filled with mostly adopted value (mode value)
    df['order_protocol'].fillna(df['order_protocol'].mode()[0], inplace=True)
    
    # Estimated Driving Duration is filled using the Median value throught our dataset and there are only 525 null values, so it will not
    # hamper the performance of the model
    df['estimated_store_to_consumer_driving_duration'].fillna(
        df['estimated_store_to_consumer_driving_duration'].median(), 
        inplace=True
    )
    
    return df

# Apply the function
df = handle_null_values(df)


/tmp/ipykernel_23/1556429825.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['store_primary_category'].fillna('Uncategorized', inplace=True)
/tmp/ipykernel_23/1556429825.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['store_primary_category'].fillna('Uncategorized', inplace=True)
/tmp/ipykernel_23/1556429825.py:7: FutureWarning:

In [9]:
df.isna().sum()

market_id                                           0
created_at                                          0
actual_delivery_time                                0
store_id                                            0
store_primary_category                              0
order_protocol                                      0
total_items                                         0
subtotal                                            0
num_distinct_items                                  0
min_item_price                                      0
max_item_price                                      0
total_onshift_dashers                           16194
total_busy_dashers                              16194
total_outstanding_orders                        16194
estimated_order_place_duration                      0
estimated_store_to_consumer_driving_duration        0
dtype: int64

----------------------------------------------------
Price Range obtained instead of Min or Max prices

In [10]:

df['price_range'] = df['max_item_price'] - df['min_item_price']

df

/tmp/ipykernel_23/3602696812.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price_range'] = df['max_item_price'] - df['min_item_price']


,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift_dashers,total_busy_dashers,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,price_range
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,1845,american,1.0,4,3441,4,557,1239,33.0,14.0,21.0,446,861.0,682
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477,mexican,2.0,1,1900,1,1400,1400,1.0,2.0,2.0,446,690.0,0
2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,5477,Uncategorized,1.0,1,1900,1,1900,1900,1.0,0.0,0.0,446,690.0,0
3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,5477,Uncategorized,1.0,6,6900,5,600,1800,1.0,1.0,2.0,446,289.0,1200
4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,5477,Uncategorized,1.0,3,3900,3,1100,1600,6.0,6.0,9.0,446,650.0,500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197423,1.0,2015-02-17 00:19:41,2015-02-17 01:24:48,2956,fast,4.0,3,1389,3,345,649,17.0,17.0,23.0,251,331.0,304
197424,1.0,2015-02-13 00:01:59,2015-02-13 00:58:22,2956,fast,4.0,6,3010,4,405,825,12.0,11.0,14.0,251,915.0,420
197425,1.0,2015-01-24 04:46:08,2015-01-24 05:36:16,2956,fast,4.0,5,1836,3,300,399,39.0,41.0,40.0,251,795.0,99
197426,1.0,2015-02-01 18:18:15,2015-02-01 19:23:22,3630,sandwich,1.0,1,1175,1,535,535,7.0,7.0,12.0,446,384.0,0


In [11]:
# Mean value is used to fill the null values in these three attributes or columns

df['total_onshift_dashers'].fillna(df['total_onshift_dashers'].mean(), inplace=True)
df['total_busy_dashers'].fillna(df['total_busy_dashers'].mean(), inplace=True)
df['total_outstanding_orders'].fillna(df['total_outstanding_orders'].mean(), inplace=True)


/tmp/ipykernel_23/646705526.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['total_onshift_dashers'].fillna(df['total_onshift_dashers'].mean(), inplace=True)
/tmp/ipykernel_23/646705526.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total_onshift_dashers'].fillna(df['total_onshift_dashers'].mean(), inplace=True)
/tmp/ipykernel_

-------------------------------------------------------
Preprocessing the date, time values present in 'created_at' and 'actual_delivery_time' attributes

In [12]:
from sklearn.preprocessing import StandardScaler, LabelEncoder

data = df.copy()

# Convert timestamps
data['created_at'] = pd.to_datetime(data['created_at'])
data['actual_delivery_time'] = pd.to_datetime(data['actual_delivery_time'])

# Extract time-based features
data['hour_of_day'] = data['created_at'].dt.hour
data['day_of_week'] = data['created_at'].dt.dayofweek
data['month'] = data['created_at'].dt.month

# Compute the delivery duration in minutes
data['delivery_duration_minutes'] = (
    (data['actual_delivery_time'] - data['created_at']).dt.total_seconds() / 60
)

# Encode categorical variables
le = LabelEncoder()
data['store_primary_category_encoded'] = le.fit_transform(data['store_primary_category'])


In [13]:
data.columns

Index(['market_id', 'created_at', 'actual_delivery_time', 'store_id',
       'store_primary_category', 'order_protocol', 'total_items', 'subtotal',
       'num_distinct_items', 'min_item_price', 'max_item_price',
       'total_onshift_dashers', 'total_busy_dashers',
       'total_outstanding_orders', 'estimated_order_place_duration',
       'estimated_store_to_consumer_driving_duration', 'price_range',
       'hour_of_day', 'day_of_week', 'month', 'delivery_duration_minutes',
       'store_primary_category_encoded'],
      dtype='object')

In [14]:

# Dashers per order available

data['dashers_per_order'] = data['total_onshift_dashers'] / (data['total_outstanding_orders'] + 1e-5)

In [15]:

# Total time from placing the order from Doordash to the point where it reaches to the consumer

data['orderPlaced_to_home_location_time'] = data['estimated_order_place_duration'] + data['estimated_store_to_consumer_driving_duration']

data.head(10)

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,...,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,price_range,hour_of_day,day_of_week,month,delivery_duration_minutes,store_primary_category_encoded,dashers_per_order,orderPlaced_to_home_location_time
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,1845,american,1.0,4,3441,4,557,...,446,861.0,682,22,4,2,62.983333,5,1.571428,1307.0
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477,mexican,2.0,1,1900,1,1400,...,446,690.0,0,21,1,2,67.066667,48,0.499998,1136.0
2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,5477,Uncategorized,1.0,1,1900,1,1900,...,446,690.0,0,20,3,1,29.683333,0,100000.000000,1136.0
3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,5477,Uncategorized,1.0,6,6900,5,600,...,446,289.0,1200,21,1,2,51.250000,0,0.499998,735.0
4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,5477,Uncategorized,1.0,3,3900,3,1100,...,446,650.0,500,2,6,2,39.833333,0,0.666666,1096.0
5,3.0,2015-01-28 20:30:38,2015-01-28 21:08:58,5477,Uncategorized,1.0,3,5000,3,1500,...,446,338.0,400,20,2,1,38.333333,0,0.999995,784.0
6,3.0,2015-01-31 02:16:36,2015-01-31 02:43:00,5477,Uncategorized,1.0,2,3900,2,1200,...,446,638.0,1500,2,5,1,26.400000,0,1.111110,1084.0
7,3.0,2015-02-12 03:03:35,2015-02-12 03:36:20,5477,Uncategorized,1.0,4,4850,4,750,...,446,626.0,1050,3,3,2,32.750000,0,0.999999,1072.0
8,2.0,2015-02-16 00:11:35,2015-02-16 00:38:01,5477,indian,3.0,4,4771,3,820,...,446,289.0,784,0,0,2,26.433333,37,0.444444,735.0
9,3.0,2015-02-18 01:15:45,2015-02-18 02:08:57,5477,Uncategorized,1.0,2,2100,2,700,...,446,715.0,500,1,2,2,53.200000,0,0.999995,1161.0


In [16]:
# Estimating the Delivery Speed and Average Delivery Time

data['Avg_delivery_time'] = data.groupby(['store_id', 'hour_of_day'])['delivery_duration_minutes'].transform('mean')

data['Delivery_Speed'] = data['Avg_delivery_time'] / (data['estimated_store_to_consumer_driving_duration'] / 60 + 1e-5)



In [17]:
data['Avg_price'] = (data['min_item_price'] + data['max_item_price'])/2
data.head()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,...,hour_of_day,day_of_week,month,delivery_duration_minutes,store_primary_category_encoded,dashers_per_order,orderPlaced_to_home_location_time,Avg_delivery_time,Delivery_Speed,Avg_price
0,1.0,2015-02-06 22:24:17,2015-02-06 23:27:16,1845,american,1.0,4,3441,4,557,...,22,4,2,62.983333,5,1.571428,1307.0,62.983333,4.389079,898.0
1,2.0,2015-02-10 21:49:25,2015-02-10 22:56:29,5477,mexican,2.0,1,1900,1,1400,...,21,1,2,67.066667,48,0.499998,1136.0,59.158333,5.144198,1400.0
2,3.0,2015-01-22 20:39:28,2015-01-22 21:09:09,5477,Uncategorized,1.0,1,1900,1,1900,...,20,3,1,29.683333,0,100000.000000,1136.0,34.008333,2.957244,1900.0
3,3.0,2015-02-03 21:21:45,2015-02-03 22:13:00,5477,Uncategorized,1.0,6,6900,5,600,...,21,1,2,51.250000,0,0.499998,735.0,59.158333,12.281981,1200.0
4,3.0,2015-02-15 02:40:36,2015-02-15 03:20:26,5477,Uncategorized,1.0,3,3900,3,1100,...,2,6,2,39.833333,0,0.666666,1096.0,33.116667,3.056920,1350.0


-----------------------------------------------------
Removing Outliers from the Dataset
---------------------------------------------------

In [18]:
def remove_outliers(df, outlier_columns):
    # Iterate over each column to remove outliers based on IQR
    for col in outlier_columns:
        # Calculate the Q1 (25th percentile) and Q3 (75th percentile)
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        
        # Calculate the IQR (Interquartile Range)
        IQR = Q3 - Q1
        
        # Define the lower and upper bounds for outliers
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Remove rows where the column values are outside the bounds
        df = df[(df[col] >= lower_bound) & (df[col] <= upper_bound)]
    
    return df

outlier_columns = [
    'subtotal', 'delivery_duration_minutes', 'num_distinct_items', 'price_range','max_item_price', 'Delivery_Speed','Avg_price'
]

data_cleaned = remove_outliers(data, outlier_columns)

# Length of the cleaned dataset after removing outliers
print(f"Original dataset size: {len(data)}")
print(f"Cleaned dataset size: {len(data_cleaned)}")


Original dataset size: 196434
Cleaned dataset size: 156055


In [19]:
data_cleaned.columns

Index(['market_id', 'created_at', 'actual_delivery_time', 'store_id',
       'store_primary_category', 'order_protocol', 'total_items', 'subtotal',
       'num_distinct_items', 'min_item_price', 'max_item_price',
       'total_onshift_dashers', 'total_busy_dashers',
       'total_outstanding_orders', 'estimated_order_place_duration',
       'estimated_store_to_consumer_driving_duration', 'price_range',
       'hour_of_day', 'day_of_week', 'month', 'delivery_duration_minutes',
       'store_primary_category_encoded', 'dashers_per_order',
       'orderPlaced_to_home_location_time', 'Avg_delivery_time',
       'Delivery_Speed', 'Avg_price'],
      dtype='object')

In [20]:
# We have the date time in numerical format and also actual delivery time in minutes
# Dropping these three columns 'created_at', 'actual_delivery_time', 'store_primary_category'

data_cleaned = data_cleaned.drop(columns=['created_at', 'actual_delivery_time', 'store_primary_category'],axis=1)

------------------------------------------------------
Model Training using XGBoost model
-----------------------------------

In [21]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tqdm import tqdm
import time

# Dataloader for our training dataset
class DeliveryDataset(Dataset):
    def __init__(self, features, target):
        self.features = features
        self.target = target
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return torch.tensor(self.features[idx], dtype=torch.float32), torch.tensor(self.target[idx], dtype=torch.float32)

# Function is used to define the features which will be used for our model training
def select_features(data):
    features = [
        'market_id',
        'store_id',
        'order_protocol',
        'total_items',
        'subtotal',
        'num_distinct_items',
        'min_item_price',
        'max_item_price',
        'total_onshift_dashers',
        'total_busy_dashers',
        'total_outstanding_orders',
        'estimated_order_place_duration',
        'estimated_store_to_consumer_driving_duration',
        'price_range',
        'hour_of_day',
        'day_of_week',
        'month',
        'store_primary_category_encoded',
        'dashers_per_order',
        'orderPlaced_to_home_location_time',
        'Avg_delivery_time',
        'Delivery_Speed',
        'Avg_price'
    ]
    
    X = data[features]
    y = data['delivery_duration_minutes']
    
    return X, y


def train_and_evaluate_model(X, y):

    # Data is split into training and testing parts
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Scaling the features
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
   # Using the Dataloader for using small batches to run our code
    train_dataset = DeliveryDataset(X_train_scaled, y_train)
    test_dataset = DeliveryDataset(X_test_scaled, y_test)
    
    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

    # Checking whether GPU is avaliable or not
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')
    
    # XGBoost Model (GPU Acceleration)
    model = xgb.XGBRegressor(
        n_estimators=100,
        learning_rate=0.1,
        random_state=42,
        tree_method='gpu_hist'
    )
    
    # Model training 
    start_time = time.time()
    for i in tqdm(range(100), desc="Training Model", unit="round"):
        model.fit(X_train_scaled, y_train, verbose=False)
        
        elapsed_time = time.time() - start_time
        estimated_total_time = elapsed_time * 100 / (i + 1)
        remaining_time = estimated_total_time - elapsed_time
        tqdm.write(f"Round {i+1} - Time elapsed: {elapsed_time:.2f}s - Estimated time left: {remaining_time:.2f}s")

    
    y_pred = model.predict(X_test_scaled)
    
    
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    

    feature_importance = pd.DataFrame({
        'feature': X.columns,
        'importance': model.feature_importances_
    }).sort_values('importance', ascending=False)
    
    return {
        'model': model,
        'mae': mae,
        'mse': mse,
        'rmse': rmse,
        'r2': r2,
        'feature_importance': feature_importance
    }



def main(df):

    # Selecting Features and Target label for our data
    X, y = select_features(df)

    # Using the function to train the model
    results = train_and_evaluate_model(X, y)
    
    # Print Results
    print("\nModel Performance Metrics:")
    print(f"Mean Absolute Error: {results['mae']:.2f}")
    print(f"Mean Squared Error: {results['mse']:.2f}")
    print(f"Root Mean Squared Error: {results['rmse']:.2f}")
    print(f"R-squared Score: {results['r2']:.2f}")
    
    # Display Feature Importance
    print("\nTop 10 Most Important Features:")
    print(results['feature_importance'].head(10))
    
    return results


results = main(data_cleaned) 


Using device: cuda


Training Model:   0%|          | 0/100 [00:00<?, ?round/s]/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:57:21] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
Training Model:   1%|          | 1/100 [00:00<01:20,  1.23round/s]

Round 1 - Time elapsed: 0.81s - Estimated time left: 80.42s


Training Model:   2%|▏         | 2/100 [00:01<01:04,  1.51round/s]

Round 2 - Time elapsed: 1.37s - Estimated time left: 66.97s


...

/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:58:15] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
Training Model:  99%|█████████▉| 99/100 [00:55<00:00,  1.67round/s]

Round 99 - Time elapsed: 55.09s - Estimated time left: 0.56s


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:58:16] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
Training Model: 100%|██████████| 100/100 [00:55<00:00,  1.80round/s]

Round 100 - Time elapsed: 55.64s - Estimated time left: 0.00s

Model Performance Metrics:
Mean Absolute Error: 7.80
Mean Squared Error: 103.25
Root Mean Squared Error: 10.16
R-squared Score: 0.50

Top 10 Most Important Features:
                                         feature  importance
20                             Avg_delivery_time    0.577487
18                             dashers_per_order    0.081316
19             orderPlaced_to_home_location_time    0.053756
12  estimated_store_to_consumer_driving_duration    0.048899
15                                   day_of_week    0.025624
16                                         month    0.024606
14                                   hour_of_day    0.019779
5                             num_distinct_items    0.018859
3                                    total_items    0.017044
8                          total_onshift_dashers    0.016808



/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:58:16] WARNING: /workspace/src/common/error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.

  warnings.warn(smsg, UserWarning)


In [22]:
print("\nModel Performance Metrics:")
print(f"Mean Absolute Error: {results['mae']:.2f}")
print(f"Mean Squared Error: {results['mse']:.2f}")
print(f"Root Mean Squared Error: {results['rmse']:.2f}")
print(f"R-squared Score: {results['r2']:.2f}")

# Display Feature Importance
print("\nTop 10 Most Important Features:")
print(results['feature_importance'].head(10))


Model Performance Metrics:
Mean Absolute Error: 7.80
Mean Squared Error: 103.25
Root Mean Squared Error: 10.16
R-squared Score: 0.50

Top 10 Most Important Features:
                                         feature  importance
20                             Avg_delivery_time    0.577487
18                             dashers_per_order    0.081316
19             orderPlaced_to_home_location_time    0.053756
12  estimated_store_to_consumer_driving_duration    0.048899
15                                   day_of_week    0.025624
16                                         month    0.024606
14                                   hour_of_day    0.019779
5                             num_distinct_items    0.018859
3                                    total_items    0.017044
8                          total_onshift_dashers    0.016808


---------------------------------------------------
Model Inference
--------------------------

In [23]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta


# Function to handle the prediction code and return the results in minutes format in which the model has been trained for delivery time
# Finally chaning it to date and time format

def predict_delivery_duration(model, X_test, y_test):
    
    # Scaling the features
    scaler = StandardScaler()
    X_test_scaled = scaler.fit_transform(X_test)
    
    # Selecting a random row for inference each time from the test data
    random_index = np.random.randint(0, len(X_test_scaled))
    sample_feature = X_test_scaled[random_index].reshape(1, -1)
    
    # Predicting delivery duration
    
    predicted_duration = float(model.predict(sample_feature)[0])
    
    # order datetime
    order_datetime = datetime.now()
    
    # Calculating the delivery datetime
    delivery_datetime = order_datetime + timedelta(minutes=predicted_duration)
    
    return {
        'selected_row_index': random_index,
        'order_datetime': order_datetime,
        'actual_delivery_duration': y_test.iloc[random_index],
        'predicted_delivery_duration': predicted_duration,
        'estimated_delivery_datetime': delivery_datetime
    }

# Main inference function
def main():
    # Select features and target from the original dataset
    features = [
        'market_id', 'store_id', 'order_protocol', 'total_items', 'subtotal', 'num_distinct_items', 'min_item_price', 'max_item_price',
        'total_onshift_dashers', 'total_busy_dashers', 'total_outstanding_orders', 'estimated_order_place_duration', 
        'estimated_store_to_consumer_driving_duration','price_range', 'hour_of_day', 'day_of_week', 'month', 
        'store_primary_category_encoded', 'dashers_per_order', 'orderPlaced_to_home_location_time', 'Avg_delivery_time', 
        'Delivery_Speed', 'Avg_price'
    ]
    
    X = data_cleaned[features]
    y = data_cleaned['delivery_duration_minutes']
    
    # Data is split into training and testing parts where 20% is for test data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Using our XGBoost trained model 
    model = results['model']
    
    inference_results = predict_delivery_duration(model, X_test, y_test)
    
    print("Selected Row Index:", inference_results['selected_row_index'])
    print("Order Datetime:", inference_results['order_datetime'].strftime('%Y-%m-%d %H:%M:%S'))
    print("Actual Delivery Duration (minutes):", inference_results['actual_delivery_duration'])
    print("Predicted Delivery Duration (minutes):", inference_results['predicted_delivery_duration'])
    print("Estimated Delivery Datetime:", inference_results['estimated_delivery_datetime'].strftime('%Y-%m-%d %H:%M:%S'))

main()

Selected Row Index: 28436
Order Datetime: 2024-12-11 16:58:16
Actual Delivery Duration (minutes): 31.966666666666665
Predicted Delivery Duration (minutes): 37.953495025634766
Estimated Delivery Datetime: 2024-12-11 17:36:13


-------------------------------------
Model Selection
------------------------

In [24]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from tqdm import tqdm
import time

# Dataloader for our training dataset
class DeliveryDataset(Dataset):
    def __init__(self, features, target):
        self.features = features
        self.target = target
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return torch.tensor(self.features[idx], dtype=torch.float32), torch.tensor(self.target[idx], dtype=torch.float32)

# Function to select features
def select_features(data):
    features = [
        'market_id', 'store_id', 'order_protocol', 'total_items', 'subtotal',
        'num_distinct_items', 'min_item_price', 'max_item_price', 'total_onshift_dashers',
        'total_busy_dashers', 'total_outstanding_orders', 'estimated_order_place_duration',
        'estimated_store_to_consumer_driving_duration', 'price_range', 'hour_of_day', 
        'day_of_week', 'month', 'store_primary_category_encoded', 'dashers_per_order', 
        'orderPlaced_to_home_location_time', 'Avg_delivery_time', 'Delivery_Speed', 'Avg_price'
    ]
    
    X = data[features]
    y = data['delivery_duration_minutes']
    
    return X, y

# Function to train and evaluate different models
def train_and_evaluate_model(X, y, model_type):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Feature Scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # Model selection
    if model_type == "xgboost":
        model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.1, random_state=42, tree_method='gpu_hist')
    elif model_type == "lightgbm":
        model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1, random_state=42)
    elif model_type == "linear":
        model = LinearRegression()
    elif model_type == "random_forest":
        model = RandomForestRegressor(n_estimators=100, random_state=42)
    elif model_type == "gradient_boosting":
        model = GradientBoostingRegressor(n_estimators=100, random_state=42)
    
    # Model training
    start_time = time.time()
    model.fit(X_train_scaled, y_train)
    
    elapsed_time = time.time() - start_time
    tqdm.write(f"Training Time for {model_type}: {elapsed_time:.2f}s")
    
    y_pred = model.predict(X_test_scaled)
    
    # Evaluation metrics
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    return {
        'model': model,
        'mae': mae,
        'mse': mse,
        'rmse': rmse,
        'r2': r2
    }


def main(df):
    
    # Selecting Features and Target label for our data
    X, y = select_features(df)

    # XGBoost Model
    print("Evaluating XGBoost Model...")
    results_xgboost = train_and_evaluate_model(X, y, "xgboost")
    print_results("XGBoost", results_xgboost)

    # LightGBM Model
    print("\nEvaluating LightGBM Model...")
    results_lgb = train_and_evaluate_model(X, y, "lightgbm")
    print_results("LightGBM", results_lgb)

    # Linear Regression Model
    print("\nEvaluating Linear Regression Model...")
    results_lr = train_and_evaluate_model(X, y, "linear")
    print_results("Linear Regression", results_lr)

    # Random Forest Regressor Model
    print("\nEvaluating Random Forest Regressor Model...")
    results_rf = train_and_evaluate_model(X, y, "random_forest")
    print_results("Random Forest", results_rf)

    # Gradient Boosting Regressor Model
    print("\nEvaluating Gradient Boosting Regressor Model...")
    results_gb = train_and_evaluate_model(X, y, "gradient_boosting")
    print_results("Gradient Boosting", results_gb)

    return results_xgboost, results_lgb, results_lr, results_rf, results_gb


def print_results(model_name, results):
    print(f"\n{model_name} Performance Metrics:")
    print(f"Mean Absolute Error: {results['mae']:.2f}")
    print(f"Mean Squared Error: {results['mse']:.2f}")
    print(f"Root Mean Squared Error: {results['rmse']:.2f}")
    print(f"R-squared Score: {results['r2']:.2f}")


results = main(data_cleaned)


Evaluating XGBoost Model...


/opt/conda/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [16:58:20] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Training Time for xgboost: 0.56s

XGBoost Performance Metrics:
Mean Absolute Error: 7.80
Mean Squared Error: 103.25
Root Mean Squared Error: 10.16
R-squared Score: 0.50

Evaluating LightGBM Model...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.004986 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3562
[LightGBM] [Info] Number of data points in the train set: 124844, number of used features: 23
[LightGBM] [Info] Start training from score 44.961240
Training Time for lightgbm: 1.08s

LightGBM Performance Metrics:
Mean Absolute Error: 7.80
Mean Squared Error: 103.10
Root Mean Squared Error: 10.15
R-squared Score: 0.50

Evaluating Linear Regression Model...
Training Time for linear: 0.14s

Linear Regression Performance Metrics:
Mean Absolute Error: 8.10
Mean Squared Error: 110.70
Root Mean Squared Error: 10.52
R-squared Score: 0.47

Eval

-------------------------------------
XGBoost and LightGBM have almost identical performance but XGBoost Performed much better among all the models in terms of training time and evaluation metrics
----------------------

--------------------------
Model Inference through XGBoost is presented above the Model Selection code
---------------------------